In [1]:
!pip install -q transformers datasets matplotlib
!!pip install -q accelerate torchinfo huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


["\x1bWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv\x1b\x1b",
 '\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b A new release of pip is available: \x1b\x1b23.0.1\x1b\x1b -> \x1b\x1b23.1.2\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b To update, run: \x1b\x1bpython -m pip install --upgrade pip\x1b']

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 7419 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7419 kB]
Fetched 7419 kB in 1s (5795 kB/s)
debconf: delaying package configuration, since apt-ut

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter
import torch 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast
from tqdm.notebook import tqdm
from transformers import T5ForConditionalGeneration, T5Config, AdamW
from tqdm.auto import tqdm
from accelerate import Accelerator

In [5]:
file_id = '1e1Tz4VycXhimx3ZyrxgL72VBaC0JP6By'
destination = 'human_proteins.tsv'
hf_name = "t5-small"
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

In [6]:
file_id = '1e1Tz4VycXhimx3ZyrxgL72VBaC0JP6By'
destination = 'human_proteins.tsv'

def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")


download_file_from_google_drive(file_id, destination)

0it [00:00, ?it/s]



saved all the data to human_proteins.tsv. total size : 33.218 MB


In [7]:
download_file_from_google_drive("1FLrC9kK5-R_NwjmX_WTqT8YqchzS9zIN", "all_proteins.tsv")

0it [00:00, ?it/s]



saved all the data to all_proteins.tsv. total size : 294.963 MB


In [8]:
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")
dataframe = dataframe[dataframe["sequence"].str.len() <= 510]
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")
dataframe = dataframe[dataframe["sequence"].str.len() <= 510]
dataframe["protein_name"] = [v.split("(")[0] for v in dataframe["protein_name"]]
dataframe["sequence"] = [" ".join(v) for v in dataframe["sequence"]]

with open("Sequences.txt", "w") as f:
    f.write("\n".join(dataframe["sequence"].values))
    
dataframe

,sequence,protein_name,length
0,M R W Q E M G Y I F Y P R K L R,Mitochondrial-derived peptide MOTS-c,16
1,M P G W F K K A W Y G L A S L L S F S S F I L ...,Clarin-2,232
2,M K R R Q K R K H L E N E E S Q E T A E K G G ...,Protein FAM170A,330
3,M D D A D P E E R N Y D N M L K M L S D L N K ...,Synaptonemal complex central element protein 3,88
4,M A F S D L T S R T V H L Y D N W I K D A D P ...,Elongation of very long chain fatty acids prot...,281
...,...,...,...
89896,M E P V D P R L E P W K H P G S Q P K T P C T ...,Protein Tat,101
89897,M G G K W S K R S A P G W N A V R E R M R R T ...,Protein Nef,206
89898,M L L L G A L L L L L A L P S Y G Q D T M Q G ...,Adiponectin,239
89899,M G Q F I S F M Q E I P I F L Q E A L N I A L ...,Pre-glycoprotein polyprotein GP complex,483


In [9]:
dataframe2 = pd.read_csv("all_proteins.tsv", sep="\t")
dataframe2 = dataframe2[dataframe2["sequence"].str.len() <= 510]
dataframe2 = dataframe2[dataframe2["sequence"].str.len() <= 510]
dataframe2["protein_name"] = [v.split("(")[0] for v in dataframe2["protein_name"]]

with open("names.txt", "w") as f:
    f.write("\n".join([v.split("(")[0].strip() for v in dataframe2["protein_name"].values]))  

dataframe2

,Unnamed: 0,sequence,protein_name,length
0,0,MRWQEMGYIFYPRKLR,Mitochondrial-derived peptide MOTS-c,16
1,1,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,Clarin-2,232
2,2,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,Protein FAM170A,330
3,3,MDDADPEERNYDNMLKMLSDLNKDLEKLLEEMEKISVQATWMAYDM...,Synaptonemal complex central element protein 3,88
4,4,MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,Elongation of very long chain fatty acids prot...,281
...,...,...,...,...
619007,529106,MNSNAPAAVIVLAAGAGTRMKSKLPKVLHEIGGRSLLMHAITAARG...,Bifunctional protein GlmU [Includes: UDP-N-ace...,492
619008,529107,MSATGSDPSRRPVDLPDLSREAVPGEKVALAPGQLQLRPTRRGKAP...,Probable dual-specificity RNA methyltransferas...,428
619009,529108,MVLASHNAKKLRELQRILAPAVPGLEAEQIVSAAGLGLPDVVEDAV...,dITP/XTP pyrophosphatase,203
619010,529109,MLPVLTADALRTAEQAHWDEHPGDDLMGRAAAEVARHATEMLGDGP...,ADP-dependent,492


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mlewand/PROT5-small", max_length=512,
                                                   padding=True)

In [11]:
ids = tokenizer.encode("M R W Q E M G Y I F Y P R K L R")
print(ids, tokenizer.decode(ids))
ids = tokenizer.encode("Protein FAM170A")
print(ids, tokenizer.decode(ids))

[1, 118, 89, 204, 116, 100, 118, 92, 131, 110, 115, 131, 88, 89, 104, 85, 89, 2] <s> M R W Q E M G Y I F Y P R K L R</s>
[1, 249, 9701, 12, 25, 2] <s> Protein FAM170A</s>


In [67]:
ids

[1, 249, 9701, 12, 25, 2]

In [12]:
_tokenized_X = []
_tokenized_Y = []

tokenized_X = []
tokenized_Y = []
sizes_X = []
sizes_Y = []

max_length_inp = 512
max_length_out = 64

for x in tqdm(dataframe["sequence"]):
    input_tokens = tokenizer.encode(x)
    tokenized_X.append(input_tokens)
    sizes_X.append(len(input_tokens))

for y in tqdm(dataframe["protein_name"]):
    out_tokens = tokenizer.encode(y)
    tokenized_Y.append(out_tokens)
    sizes_Y.append(len(out_tokens))
    
for x,y in tqdm(zip(_tokenized_X, _tokenized_Y)):
   if len(x) <= max_length_inp and len(y) <= max_length_out:
    tokenized_X.append(x)
    tokenized_Y.append(y)

  0%|          | 0/89901 [00:00<?, ?it/s]

  0%|          | 0/89901 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [13]:
train_val_split = 0.9
l = len(tokenized_X)
split_idx = int(train_val_split * l)

idxes = np.arange(l)
r = np.random.permutation(idxes)

train_idxs = r[:split_idx]
val_idxs = r[split_idx:]

train_X, train_Y = [], []
val_X, val_Y = [], []


for idx in train_idxs:
    train_X.append(tokenized_X[idx])
    train_Y.append(tokenized_Y[idx])
    
for idx in val_idxs:
    val_X.append(tokenized_X[idx])
    val_Y.append(tokenized_Y[idx])   
    
print(f"train sizes ;  X : {len(train_X)}, Y : {len(train_Y)}")
print(f"val sizes ;  X : {len(val_X)}, Y : {len(val_Y)}")


train sizes ;  X : 80910, Y : 80910
val sizes ;  X : 8991, Y : 8991


In [14]:
class CustomAADataset(Dataset):
  def __init__(self, X, Y, max_length=512, max_length2=32):
    self.max_length = max_length
    self.max_length2 = max_length2 
    self.X = X
    self.Y = Y
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    input_tokens = self.X[idx]
    output_tokens = self.Y[idx]

    # Pad input and output tokens
    input_tokens = input_tokens + [0] * (self.max_length - len(input_tokens))

    output_tokens = output_tokens + [0] * (self.max_length2 - len(output_tokens))

    return {"input_ids": torch.tensor(input_tokens, dtype=torch.long),
            "labels": torch.tensor(output_tokens, dtype=torch.long),
            }


In [15]:
# Load data
from datasets import Dataset as Dataset_hf

dataset = Dataset_hf.from_dict({"source": dataframe["sequence"].values, "target": dataframe["protein_name"].values})

In [16]:
train_dataset = CustomAADataset(train_X, train_Y, max_length_inp, max_length_out)
val_dataset = CustomAADataset(val_X, val_Y, max_length_inp, max_length_out)

train_dataset[0]

{'input_ids': tensor([  1, 118,  99,  99,  90,  92,  87, 108, 100, 110,  92, 104,  85, 115,
         101,  92,  92,  85, 108, 204,  90, 109, 109, 116, 100, 109,  85,  89,
          90, 131, 115,  90, 116, 131,  92, 100, 101, 101, 108, 111, 101, 110,
         118, 104, 108, 104, 109, 109,  99, 116,  90,  89,  92, 115,  92, 115,
         101, 104, 115, 104, 108,  88,  99, 111, 101,  92, 109, 101,  85,  87,
          90,  89,  88, 129, 109,  85, 108,  92,  89,  99, 110, 108,  88, 104,
          88, 111, 109,  88,  89,  92, 118, 116,  88, 100,  89, 109,  89,  88,
         104, 100,  92, 204, 116, 104,  92,  88,  89,  90, 108,  99,  90, 104,
          90,  99, 104, 110, 115, 101,  92,  92, 110,  88, 129,  99, 111,  92,
         100, 109, 100,  85,  89, 100, 131, 115, 104, 104, 115,  92, 101, 101,
         109, 100, 101, 101, 118, 110, 131, 108,  87, 100, 104, 116,  89,  88,
          89,  92, 115,  92, 115, 110, 109, 115, 100, 108, 100, 116,  90, 101,
         108, 116,  87, 101,  99, 118, 

In [25]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=None, padding=True, max_length=512)

model_name = "t5-small"
config = T5Config.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True)

In [26]:
dats = None
eval_d = None

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    global dats
    pred_ids = torch.argmax(logits[0],dim=-1)
    return pred_ids

def compute_metrics(pred):
    global eval_d
    eval_d = pred
    labels = pred.label_ids
    # labels = np.argmax(labels, axis=1)
    preds = pred.predictions

    tot = np.sum(labels != 0) + 1e-10
    classified_correctly = 0
    exact_matches = 0
    for i in range(preds.shape[0]):
        mask = preds[i, :] != 0
        classified_correctly += np.sum(preds[i, mask] == labels[i, mask])
        exact_matches += np.all(preds[i,:] == labels[i, :])
            
    return {"matches": classified_correctly / tot, "exact_matches" : exact_matches / preds.shape[0]}

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="PROT5-small",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    num_train_epochs=30,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    fp16=True,  # Enable mixed precision training if your GPU supports it
    push_to_hub=True,
    hub_model_id="mlewand/PROT5-small-v2",
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="epoch"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics= preprocess_logits_for_metrics
)

trainer.train()

/workspace/PROT5-small is already a clone of https://huggingface.co/mlewand/PROT5-small-v2. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Matches,Exact Matches
1,0.327100,0.254362,0.705893,0.453898
2,0.243700,0.222853,0.737829,0.462351
3,0.215800,0.201158,0.755404,0.478812
4,0.199100,0.190414,0.768285,0.503392
5,0.187000,0.180720,0.777898,0.516294
6,0.177700,0.172958,0.786376,0.535536
7,0.170200,0.168122,0.792378,0.543099
8,0.163700,0.164963,0.797606,0.552664
9,0.158100,0.159285,0.801304,0.559671
10,0.153300,0.157703,0.806996,0.568902


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=75870, training_loss=0.1664552430413969, metrics={'train_runtime': 10659.2535, 'train_samples_per_second': 227.718, 'train_steps_per_second': 7.118, 'total_flos': 3.285151547129856e+17, 'train_loss': 0.1664552430413969, 'epoch': 30.0})

In [100]:
def decode(pred):
    mask = pred != 0
    return tokenizer.decode(pred[mask][1:-1])
    
model = model.to("cpu")
val_dl = DataLoader(val_dataset, batch_size=1, shuffle=True)

exact_match = 0
i = 0

with tqdm(val_dl) as pb:
    for batch in val_dl:
        ret = model(**batch)
        pred = torch.argmax(ret.logits, axis=-1).reshape(-1)
    
        exact_match += torch.all(pred == batch['labels']).item()
        i += 1
        # print(decode(pred), decode(batch['labels']))
        pb.update(1)
        pb.set_description(f"{exact_match}/{i} ; Acc : {round( 100. * exact_match / i , 2)}")

print(f"{exact_match}/{i}")

  0%|          | 0/8991 [00:00<?, ?it/s]

5377/8991


In [102]:
tokenizer.push_to_hub("mlewand/PROT5-small-v2")
#model.push_to_hub("mlewand/PROT5-small-v2")


CommitInfo(commit_url='https://huggingface.co/mlewand/PROT5-small-v2/commit/b9b96bdfe83002baad22a882a4519aad615e70a4', commit_message='Upload tokenizer', commit_description='', oid='b9b96bdfe83002baad22a882a4519aad615e70a4', pr_url=None, pr_revision=None, pr_num=None)